In [1]:
from keras.applications import VGG16
import os

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
base_dir = '/home/camaro/Desktop/pneumonia_detec/chest_xray/chest_xray'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

Using TensorFlow backend.


In [2]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])


In [3]:
model.load_weights('pneumonia_finetune_vgg16.h5')

In [4]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input

In [7]:
# pic_normal = os.path.join(base_dir, 'train/NORMAL/IM-0151-0001.jpeg')
# pic_abnormal = os.path.join(base_dir, 'train/PNEUMONIA/person1_bacteria_1.jpeg')

pic_normal = '/home/camaro/Desktop/pneumonia_detec/chest_xray/chest_xray/train/NORMAL/IM-0115-0001.jpeg'
pic_abnormal = '/home/camaro/Desktop/pneumonia_detec/chest_xray/chest_xray/train/PNEUMONIA/person1_bacteria_2.jpeg'

image = load_img(pic_abnormal, target_size=(150, 150))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)

In [8]:
yhat

array([[0.]], dtype=float32)